In [1]:
from IPython.display import display, Markdown, Latex
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings('ignore')
import csv
import numpy as np
import sys, getopt
import math
import random
import timeit
import os

In [72]:
######## FUNCTIONS

### Functions to process the files and print in log
def processFile(filename):
    '''No final, vamos ter:
    nElements => numero de bins
    capacity1 => capacidade 1 das bins
    capacity2 => capacidade 2 das bins
    items => dicionario com:
            chaves = numero do item
            valores = lista das capacidades usadas pelo item
            
    exemplo : processFile('instances/CL_01_25_01.TXT') dá:
    nElements = 25
    capacity1 = 1000 = capacity2
    items = {1: [165, 125], 2: [315, 355], 3: [117, 321],..., 24: [125, 111], 25: [136, 220]}
    '''   
    print('Start processing file :', filename)
    instance = filename[:-3]
    delimiter = ' '
    with open(filename,'r') as file:
        csv_reader = csv.reader(file, delimiter = delimiter)
        nElements, capacity1, capacity2 = 0,0,0 # general data about number of items and capacities of bins
        items = {} # dictionary to store the items data
        lineCount = 0  # for parsing
        for row in csv_reader:
            if lineCount == 0 : # nElements
                for i in range(0,len(row)):
                    if row[i] != '': # number of blank spaces is not fixed
                        nElements = int(row[i])
            elif lineCount == 1: # capacity1 and capacity2
                capacitiesCount = 0 # for parsing
                for i in range(0,len(row)):
                    if row[i] != '': 
                        if capacitiesCount == 0:
                            capacity1 = int(row[i])
                            capacitiesCount += 1
                        else:
                            capacity2 = int(row[i])
            else: # take data from each item
                itemCapacity1 = 0 # capacity 1 of item
                itemCapacity2 = 0 # capacity 2 of item
                itemsCapacitiesCount = 0 # for parsing
                for i in range(0,len(row)):
                    if row[i] != '': 
                        if itemsCapacitiesCount == 0:
                            itemNumber = int(row[i])
                            itemsCapacitiesCount += 1
                        elif itemsCapacitiesCount == 1:
                            itemCapacity1 = int(row[i])
                            itemsCapacitiesCount += 1
                        else:
                            itemCapacity2 = int(row[i])
                items[itemNumber] = [itemCapacity1,itemCapacity2]        
            lineCount += 1
    print('nElements : ',nElements)
    print('capacity1 : ',capacity1)
    print('capacity2 : ',capacity2)
    print('items : ',items)
    print('Finished processing file :', filename)
    print('\n')
    return nElements, capacity1, capacity2, items
 

def getBasicSolution(minNumBins):
    '''
    Will put one item per bin (so there will be nElements bins)
    At the end, we will have:
    bins = {1: [1], 2: [2], 3: [3],..., nElements: [nElements]}
    binsCapacities1 = {1: [bin1Capacity1, 0], 2: [bin2Capacity1, 0],..., nElements: [binnElementsCapacity1, 0]}
    binsCapacities2 = {1: [bin1Capacity2, 0], 2: [bin2Capacity2, 0],..., nElements: [binnElementsCapacity2, 0]}
    The 0 at the end of each list represents the extra capacity used by the bin
    '''
    print('First solution :')
    bins, binsCapacities1, binsCapacities2 = {}, {}, {}
    for itemNumber, itemCapacities in items.items():
        bins[itemNumber] = [itemNumber]
        binsCapacities1[itemNumber] = [itemCapacities[0],0]
        binsCapacities2[itemNumber] = [itemCapacities[1],0]
        
    print('minNumBins : ', minNumBins)
    print('currentBins : ', bins)
    print('currentBinsCapacities1 : ', binsCapacities1)
    print('currentBinsCapacities2 : ', binsCapacities2)
    print('\n')
    
    return bins, binsCapacities1, binsCapacities2

def relocate(bins, binsCapacities1, binsCapacities2, binOrigin, binDestination, item):
    '''Moves item from binOrigin to binDestination'''
    bins[binOrigin].remove(item) # works because item is only once in bins[bin2]
    bins[binDestination].append(item) 
    itemCapacity1, itemCapacity2 = items[item][0], items[item][1]
    binsCapacities1[binDestination][0] += itemCapacity1
    binsCapacities2[binDestination][0] += itemCapacity2
    binsCapacities1[binOrigin][0] -= itemCapacity1
    binsCapacities2[binOrigin][0] -= itemCapacity2
    #print(bins, binsCapacities1, binsCapacities2)
    
def swap(bins, binsCapacities1, binsCapacities2, nBin1, nBin2, item1, item2):
    '''Exchanges item1 from bin1 with item2 from bin2'''
    relocate(bins, binsCapacities1, binsCapacities2, nBin1, nBin2, item1)
    relocate(bins, binsCapacities1, binsCapacities2, nBin2, nBin1, item2)
       
def isRelocatePossible(items, bins, binsCapacities1, binsCapacities2, nBin2, item):
    '''Verifies if none of the capacities of bin2 will achieve the limit if inserting item in it'''
    itemCapacity1, itemCapacity2 = items[item][0], items[item][1]
    return ((binsCapacities1[nBin2][0] + itemCapacity1 <= capacity1) & (binsCapacities2[nBin2][0] + itemCapacity2 <= capacity2))
    
def isSwapPossible(items, bins, binsCapacities1, binsCapacities2, nBin1, nBin2, item1, item2):
    '''Verifies if none of the capacities of bin1 or bin2 will achieve the limit if inserting item1
    in bin2 or item2 in bin1'''
    item1Capacity1, item1Capacity2 = items[item1][0], items[item1][1]
    item2Capacity1, item2Capacity2 = items[item2][0], items[item2][1]
    
    return ((binsCapacities1[nBin2][0] + item1Capacity1 <= capacity1) & (binsCapacities2[nBin2][0] + item1Capacity2 <= capacity2) 
           & (binsCapacities1[nBin1][0] + item2Capacity1 <= capacity1) & (binsCapacities2[nBin1][0] + item2Capacity2 <= capacity2))  

def evaluateRelocate(items, bins, binsCapacities1, binsCapacities2, nBin1, nBin2, item):
    '''Gives the amount of empty space (sum of both capacities) we would gain if relocating item from bin1 in bin2
    ATTENTION : ONLY USE IF BIN1 HAS EXTRACAPACITY1 > 0 OR EXTRACAPACITY2 > 0'''
    if (isRelocatePossible(items, bins, binsCapacities1, binsCapacities2, nBin2, item)):
        capacity1GoingOut = min(items[item][0], binsCapacities1[nBin1][1]) # binsCapacities1[1] = capacity1EXTRA
        capacity2GoingOut = min(items[item][0], binsCapacities2[nBin1][1]) # binsCapacities2[1] = capacity2EXTRA
        return  capacity1GoingOut + capacity2GoingOut
    else:
        return 0  
    
def evaluateSwap(items, bins, binsCapacities1, binsCapacities2, nBin1, nBin2, item1, item2):
    '''Evaluates if swap between elements item1 (from bin1) and item2 (from bin2) will decrease the empty space (sum of both capacities)
    ATTENTION : ONLY USE IF BIN1 HAS EXTRACAPACITY1 > 0 OR EXTRACAPACITY2 > 0'''
    if (isSwapPossible(items, bins, binsCapacities1, binsCapacities2, nBin1, nBin2, item1, item2)):
        capacity1GoingOut = min(items[item1][0], binsCapacities1[nBin1][1]) # binsCapacities1[1] = capacity1EXTRA
        capacity2GoingOut = min(items[item1][0], binsCapacities2[nBin1][1]) # binsCapacities2[1] = capacity2EXTRA
        return  capacity1GoingOut + capacity2GoingOut
    else:
        return 0 
    
def evaluateAllMoves(items, bins, binsCapacities1, binsCapacities2, nLastBin):
    ''''''
    '''foundMove = False
    try all
        if best : foundMove = True, return foundMove
    return foundMove'''
    
    foundMove = False
    bestMoveInfo = []
    bestImprovement = 0
    # Relocate
    for item in bins[nLastBin]:
        for binDestination in range(1,nLastBin):
            relocateImprovement = evaluateRelocate(items, bins, binsCapacities1, binsCapacities2, nLastBin, binDestination, item)
            if (relocateImprovement > bestImprovement):
                bestMoveInfo = ['R', binDestination, item]
                bestImprovement = relocateImprovement
                foundMove = True
    # Swap
    for item1 in bins[nLastBin]:
        for binDestination in range(1,nLastBin):
            for item2 in bins[binDestination]:
                swapImprovement = evaluateSwap(items, bins, binsCapacities1, binsCapacities2, nLastBin, binDestination, item1, item2)
                if (swapImprovement > bestImprovement):
                    bestMoveInfo = ['S', binDestination, item1, item2]
                    bestImprovement = relocateImprovement
                    foundMove = True
                    
    return foundMove, bestMoveInfo, bestImprovement

def localSearch(items, bins, binsCapacities1, binsCapacities2, nLastBin):
    '''   '''
    '''bestMoveType = 
     bestMoveInfo = 
    isThereBestMove = evaluateAllMoves()  => just evaluating !!
    if best move : doIt
    else : return
    
    '''
    foundMove, bestMoveInfo, bestImprovement = evaluateAllMoves(items, bins, binsCapacities1, binsCapacities2, nLastBin)
    if foundMove == True:
        if bestMoveInfo[0] == 'R':
            relocate(bins, binsCapacities1, binsCapacities2, nLastBin, bestMoveInfo[1], bestMoveInfo[2])
        else:
            swap(bins, binsCapacities1, binsCapacities2, nLastBin, bestMoveInfo[1], bestMoveInfo[2], bestMoveInfo[3])
        return True
    else:
        return False
    #relocate(currentBins, currentBinsCapacities1, currentBinsCapacities2, 1,2,1)
    #swap(currentBins, currentBinsCapacities1, currentBinsCapacities2, 3, 4, 3, 4)
    
def geneticAlgo():
    '''   '''
    return 3
    
def mcbpp(filename):
    '''Implements mcbpp and tries to give the best solution (minimal number of bins)'''
    global nElements, capacity1, capacity2, items
    nElements, capacity1, capacity2, items = processFile(filename)
    
    # Variables
    minNumBins = nElements  # Parameter to minimize
    currentBins, currentBinsCapacities1, currentBinsCapacities2 = getBasicSolution(minNumBins) # Dictionary with current items in bins (len(currentBins) = minNumBins)
    
    # Minimizing minNumBins
    viableSolution = True
    while (viableSolution):
        viableSolutionFound = localSearch(items, currentBins, currentBinsCapacities1, currentBinsCapacities2, minNumBins)
        #viableSolutionFound = geneticAlgo()
        print('viableSolutionFound : ',viableSolutionFound)
        # da false quando nao encotramos solucao viavel 
        if viableSolutionFound == True:
            minNumBins -= 1
        else:
            viableSolution = False
        viableSolution = False # for debug  
        
    print(evaluateRelocate(items, currentBins, currentBinsCapacities1, currentBinsCapacities2, 4, 2, 4))
    print('Final solution :')
    print('minNumBins : ', minNumBins)
    print('currentBins : ', currentBins)
    print('currentBinsCapacities1 : ', currentBinsCapacities1)
    print('currentBinsCapacities2 : ', currentBinsCapacities2)
    
    
if __name__ == "__main__":
    mcbpp('instances/CL_01_25_01.TXT')

Start processing file : instances/CL_01_25_01.TXT
nElements :  25
capacity1 :  1000
capacity2 :  1000
items :  {1: [165, 125], 2: [315, 355], 3: [117, 321], 4: [137, 188], 5: [143, 224], 6: [301, 195], 7: [278, 269], 8: [139, 110], 9: [353, 388], 10: [285, 367], 11: [303, 314], 12: [272, 297], 13: [160, 182], 14: [334, 236], 15: [359, 225], 16: [203, 315], 17: [232, 175], 18: [313, 169], 19: [113, 166], 20: [298, 131], 21: [135, 132], 22: [191, 334], 23: [232, 105], 24: [125, 111], 25: [136, 220]}
Finished processing file : instances/CL_01_25_01.TXT


First solution :
minNumBins :  25
currentBins :  {1: [1], 2: [2], 3: [3], 4: [4], 5: [5], 6: [6], 7: [7], 8: [8], 9: [9], 10: [10], 11: [11], 12: [12], 13: [13], 14: [14], 15: [15], 16: [16], 17: [17], 18: [18], 19: [19], 20: [20], 21: [21], 22: [22], 23: [23], 24: [24], 25: [25]}
currentBinsCapacities1 :  {1: [165, 0], 2: [315, 0], 3: [117, 0], 4: [137, 0], 5: [143, 0], 6: [301, 0], 7: [278, 0], 8: [139, 0], 9: [353, 0], 10: [285, 0], 11

In [70]:
if __name__ == "__main__":
    mcbpp('instances/CL_01_25_01.TXT')

Start processing file : instances/CL_01_25_01.TXT
nElements :  25
capacity1 :  1000
capacity2 :  1000
items :  {1: [165, 125], 2: [315, 355], 3: [117, 321], 4: [137, 188], 5: [143, 224], 6: [301, 195], 7: [278, 269], 8: [139, 110], 9: [353, 388], 10: [285, 367], 11: [303, 314], 12: [272, 297], 13: [160, 182], 14: [334, 236], 15: [359, 225], 16: [203, 315], 17: [232, 175], 18: [313, 169], 19: [113, 166], 20: [298, 131], 21: [135, 132], 22: [191, 334], 23: [232, 105], 24: [125, 111], 25: [136, 220]}
Finished processing file : instances/CL_01_25_01.TXT


First solution :
minNumBins :  25
currentBins :  {1: [1], 2: [2], 3: [3], 4: [4], 5: [5], 6: [6], 7: [7], 8: [8], 9: [9], 10: [10], 11: [11], 12: [12], 13: [13], 14: [14], 15: [15], 16: [16], 17: [17], 18: [18], 19: [19], 20: [20], 21: [21], 22: [22], 23: [23], 24: [24], 25: [25]}
currentBinsCapacities1 :  {1: [165, 0], 2: [315, 0], 3: [117, 0], 4: [137, 0], 5: [143, 0], 6: [301, 0], 7: [278, 0], 8: [139, 0], 9: [353, 0], 10: [285, 0], 11

In [4]:
dic = {1: [0], 2: [2], 3: [3], 25: [25]}
print(dic[1])                                                                                           

[0]


KeyError: 0